# Implement the node class
* Define the data type of `payload` which is the pre-defined input parameter of `run` method.
* Define the output type of `run` method.

In [1]:
from dataclasses import dataclass

from pipesche import BaseNode


@dataclass
class AddPayload:
    x: int
    y: int


class Add(BaseNode[AddPayload, int]):
    def run(self, payload: AddPayload) -> int:
        result = payload.x + payload.y
        print(f"{payload.x} + {payload.y} = {result}")
        return result


class PrintSomeThing(BaseNode[None, None]):
    def run(self, payload: None) -> None:
        print("Some thing...")

# Construct the pipeline
* Add the node to pipeline.
* Link the nodes in the pipeline and define the input source.

In [6]:
from pipesche import Pipeline


node_1 = Add("node_1")
node_2 = PrintSomeThing("node_2")
node_3 = Add("node_3")

pipeline = Pipeline()
pipeline.add_node(node_1, input_key="input_1")
pipeline.add_node(node_2)
pipeline.add_node(node_3, input_key="input_2")
pipeline.connect(node_1.name, node_3.name)
pipeline.connect(node_3.name, node_2.name)

pipeline.run({"input_1": AddPayload(1, 2), "input_2": AddPayload(3, 4)})

# Output:
# 1 + 2 = 3
# 3 + 4 = 7
# Some thing...

1 + 2 = 3
3 + 4 = 7
Some thing...


# Get result of the node

In [7]:
pipeline.get_data(key=node_3.name)

# Output:
# 7

7

# What if our node's input comes from other nodes outputs?
* Set the input key of the node to the name of output key of other node. (Default output key of the node is its own name)

In [3]:
class Square(BaseNode[int, int]):
    def run(self, payload: int) -> int:
        result = payload ** 2
        print(f"{payload}^2 = {result}")
        return result

node_1 = Square("node_1")
node_2 = Square("node_2")
node_3 = Square("node_3")

pipeline = Pipeline()
pipeline.add_node(node_1, input_key="input_1")
pipeline.add_node(node_2, input_key=node_1.name)
pipeline.add_node(node_3, input_key="input_2")
pipeline.connect(node_1.name, node_2.name)
pipeline.connect(node_2.name, node_3.name)

pipeline.run({"input_1": 2, "input_2": 3})

# Output:
# 2^2 = 4
# 4^2 = 16
# 3^2 = 9

2^2 = 4
4^2 = 16
3^2 = 9


# What if our node's input is combined from multiple nodes outputs or source?
* Define the function to merge the inputs from multiple source.

In [5]:
node_1 = Add("node_1")
node_2 = Add("node_2")
node_3 = Square("node_3")

pipeline = Pipeline()
pipeline.add_node(node_1, input_key="input_1")
pipeline.add_node(node_2, input_key="merge.node_1.node_3")
pipeline.add_node(node_3, input_key="input_2")
pipeline.connect(node_1.name, node_2.name)
pipeline.connect(node_3.name, node_2.name)

def merge_func(x: int, y: int) -> AddPayload:
    return AddPayload(x, y)

pipeline.merge_data(
    keys=["node_1", "node_3"],
    new_key="merge.node_1.node_3",
    merge_function=merge_func
)

pipeline.run({"input_1": AddPayload(1, 2), "input_2": 3})

# Output:
# 1 + 2 = 3
# 3^2 = 9
# 3 + 9 = 12

1 + 2 = 3
3^2 = 9
3 + 9 = 12
